## Detect Medicare Fraud in the US using Classification

Medical ID Fraud is a prevalent but not often talked about issue in the US.

Forbes estimates that fraudulent medical accounts make up 3-10% of the entire multi-billion US healthcare system every year.

- Combining all years for CMS (2012-2017) and LEIE (01/2018-12/2019).


This investigation follows Johnson & Khoshgoftaar in cleaning and aggregating data. I used 2 main datasets including 

<a id='menu'></a>
### Menu

- <a href='#cms'>1. Pull CMS Data</a>

- <a href='#leie'>2. Pull LEIE Data</a>

- <a href='#combined'>3. Combine and Label Fraud</a>

- <a href='#visualization'>4. Build Visualizations</a>
    - <a href='#usmap'>US Medicare Fraud Map</a>
    - <a href='#histogram1'>Histograms of Submitted Charge/Mediacre Reimbursements Comparing Fraud vs. Non-Fraud</a>
    - <a href='#correlation1'>Correlation Heatmap for Continuous Variables </a>
    - <a href='#correlation2'>Table of High Correlations</a>
    - <a href='#barchart1'>Top 15 Fraudulent Medicare Category by Service Count</a>
    - <a href='#barchart2'>Top 15 Fraudulent Medicare Category by Service Count & Gender</a>

- <a href='#train'>5. Prep Data for Training</a>
    - <a href='#norm'>Choose Features & One-hot Encoding</a>
    - <a href='#ros'>Random Undersampling</a>

In [1]:
import os
import pandas as pd

In [2]:
# Set up data directory
CWD = os.getcwd()
cms_data_dir = os.path.join(CWD, 'CMSData')

In [3]:
# Some years columns are capitalized and other years the columns are lowercase:
capitalization_dict = {
    '2012': str.upper,
    '2013': str.upper,
    '2014': str.lower,
    '2015': str.lower,
    '2016': str.upper,
    '2017': str.lower,
}

<a href='#menu'>[Menu]</a>
<a id='cms'></a>

### 1. CMS Part B dataset

In [4]:
# Set dtypes based on https://www.cms.gov/Research-Statistics-Data-and-Systems/Statistics-Trends-and-Reports/...
#Medicare-Provider-Charge-Data/Physician-and-Other-Supplier2017
partB_dtypes = {
    'npi': 'str',
    'nppes_provider_last_org_name': 'str',
    'nppes_provider_first_name': 'str',
    'nppes_provider_mi': 'str',
    'nppes_credentials': 'str',
    'nppes_provider_gender': 'str',
    'nppes_entity_code': 'str',
    'nppes_provider_street1': 'str',
    'nppes_provider_street2': 'str',
    'nppes_provider_city': 'str',
    'nppes_provider_zip': 'str',
    'nppes_provider_state': 'str',
    'nppes_provider_country': 'str',
    'provider_type': 'str',
    'medicare_participation_indicator': 'str',
    'place_of_service': 'str',
    'hcpcs_code': 'str',
    'hcpcs_description': 'str',
    'hcpcs_drug_indicator': 'str',
    'line_srvc_cnt': 'float64',
    'bene_unique_cnt': 'float64',    
    'bene_day_srvc_cnt': 'float64',
    'average_medicare_allowed_amt': 'float64',
    'average_submitted_chrg_amt': 'float64',
    'average_medicare_payment_amt': 'float64',
    'average_medicare_standard_amt': 'float64',
}

In [5]:
# Get dfs for all years - TAKE A FEW MINUTES
years = ['2012','2013','2015','2016']
dfs   = []

for year in years:
    file = os.path.join(cms_data_dir, f'cms{year}.txt')
    dtypes = dict(zip(list(map(capitalization_dict[year], partB_dtypes.keys())), list(partB_dtypes.values()))) #get correct column capitalization and dtype
    df = pd.read_csv(file, delimiter='\t', dtype=dtypes)
    df.columns = map(str.lower, df.columns)  # make all variable names lowercase
    df['year'] = year #add Year column 
    dfs.append(df)

In [6]:
# Concatenate
partB_df = pd.concat(dfs, axis=0, ignore_index=True, sort=False)
partB_df.shape

(37653939, 30)

In [ ]:
# Remove rows corresponding to drugs because LINE_SRVC_CNT for them is not a desirable count
partB_df = partB_df[(partB_df['hcpcs_drug_indicator'] == 'N')]
partB_df.shape

In [ ]:
# Drop missing NPI and HCPCS - "Medicare fraud detection using neural networks" (Johnson, Khoshgoftaar 2019)
# This means dropping 2014 and 2016 - both did not have HCPCS Code
partB_df = partB_df.dropna(subset = ['npi','hcpcs_code'])
partB_df.shape

In [ ]:
# Keep variables based on "Medicare fraud detection using neural networks" (Johnson, Khoshgoftaar 2019)
partB_variables_to_keep = [
    'npi',
    'provider_type',
    'nppes_provider_city', # keep
    'nppes_provider_zip', # keep
    'nppes_provider_state', # keep
    'nppes_provider_country', # keep
    'hcpcs_code',  # not in paper but kept
    'hcpcs_description',  # not in paper but kept
    'hcpcs_drug_indicator',  # not in paper but kept
    'place_of_service',  # not in paper but kept
    'nppes_provider_gender',
    'line_srvc_cnt',
    'bene_unique_cnt',
    'bene_day_srvc_cnt',
    'average_submitted_chrg_amt',
    'average_medicare_payment_amt',
    'year' # need Year for labeling
]
partB_df = partB_df[partB_variables_to_keep]

In [ ]:
partB_df.head()

In [ ]:
partB_df.loc[partB_df['npi'] == '1003000142'][['npi',
                                             'provider_type',
                                             'place_of_service',
                                             'line_srvc_cnt',
                                             'average_submitted_chrg_amt',
                                             'year']][:5]

In [ ]:
partB_df['year'].value_counts()

In [ ]:
# Write all combined CMS to csv
#partB_df.to_csv('combined-partB-data-v2')

<a href='#menu'>[Menu]</a>
<a id='leie'></a>

### 2. LEIE Dataset

In [ ]:
leie_data_dir = os.path.join(CWD, 'LEIEData')

In [ ]:
leie_dtypes = {
    'LASTNAME': 'str',
    'FIRSTNAME': 'str',
    'MIDNAME': 'str',
    'BUSNAME' : 'str',
    'GENERAL': 'str',
    'SPECIALTY': 'str',
    'UPIN': 'str',
    'NPI': 'str',
    'DOB': 'str',
    'ADDRESS': 'str',
    'CITY': 'str',
    'STATE': 'str',
    'ZIP': 'str',
    'EXCLTYPE': 'str',
    'EXCLDATE': 'int64',
    'REINDATE': 'int64',
    'WAIVERDATE': 'int64',
    'WVRSTATE': 'str',
}

In [ ]:
#LEIE data is monthly between 01/2018 (1801) - 12/2019 (1912)
year_months = ['1801','1802','1803','1804','1805','1806','1807','1808','1809','1810','1811','1812',
            '1901','1902','1903','1904','1905','1906','1907','1908','1909','1910','1911','1912']
dfs = []

for year_month in year_months:
    file = os.path.join(leie_data_dir, f'leie{year_month}-excl.csv')
    df   = pd.read_csv(file, dtype=leie_dtypes)
    df.columns = map(str.lower, df.columns)
    dfs.append(df)

In [ ]:
# Concatenate
leie_df = pd.concat(dfs, axis=0, ignore_index=True, sort=False)
leie_df.shape

In [ ]:
leie_df.head()

In [ ]:
# Drop NPI = 0, which means missing - A LOT ARE MISSING, which is a problem for the data
leie_df = leie_df[leie_df['npi'] != 0]
leie_df.shape

In [ ]:
# Keep exclusions most related to Fraud
exclusions_to_keep = [
    '1128a1',
    '1128a2',
    '1128a3',
    '1128b4',
    '1128b7',
    '1128c3Gi',
    '1128c3gii',
]
leie_df = leie_df[leie_df['excltype'].isin(exclusions_to_keep)]
leie_df.shape

In [ ]:
leie_df['excltype'].value_counts()

- 1128a1: Conviction of program-related crimes
- 1128a2: Conviction of relating to patient abuse or neglect
- 1128a3: Felony conviction relating to healthcare fraud
- 1128a4: License revocation, suspension, or surrender
- 1128a3: Fraud, kickbacks, other prohibited activities

In [ ]:
# Write all combined LEIE to csv
#partB_df.to_csv('combined-leie-data')

<a href='#menu'>[Menu]</a>
<a id='combined'></a>

### 3. Combine/Label Data

In [ ]:
from datetime import datetime, timedelta
import numpy as np

In [ ]:
# Convert to datetime
leie_df['excldate'] = pd.to_datetime(leie_df['excldate'], format='%Y%m%d', errors ='ignore')

In [ ]:
# Round excl date to the nearest year Johnson & Khoshgoftaar (2019)
def round_to_year(dt=None):
    year = dt.year
    month = dt.month
    if month >= 6:
        year = year + 1
    return datetime(year=year,month=1,day=1)

leie_df['excl_year'] = leie_df.excldate.apply(lambda x: round_to_year(x))

In [ ]:
# Make exclusion dict 
# 1215053665 has 2 exclusions, so sort also df to get latest year
excl_year_dict = dict([npi, year] for npi, year in zip(leie_df.sort_values(by='excl_year').npi, leie_df.sort_values(by='excl_year').excl_year))

In [ ]:
# Get label as 0 or 1
partB_df['excl_year'] = partB_df['npi'].map(excl_year_dict)
partB_df['excl_year'] = partB_df['excl_year'].fillna(datetime(year=1900,month=1,day=1)) # fill NaN, physicians without exclusion, with year 1900

partB_df['year'] = pd.to_datetime(partB_df['year'].astype(str), format='%Y', errors ='ignore')
partB_df['fraudulent'] = np.where(partB_df['year'] < partB_df['excl_year'], 1, 0) # compare year vs. exclusion year to get Fraudulent

In [ ]:
print("partB_df is our combined dataset with shape: {0}".format(partB_df.shape))

<a href='#menu'>[Menu]</a>
<a id='visualization'></a>

### 4. Draw Visualizations

In [ ]:
%matplotlib inline

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.figure_factory as ff
import plotly.graph_objects  as go
from plotly.subplots import make_subplots

In [ ]:
# Get number and amount of fraudulent services
partB_df['fraud_line_srvc_cnt'] = partB_df['line_srvc_cnt']*partB_df['fraudulent']
partB_df['fraud_average_submitted_chrg_amt'] = partB_df['average_submitted_chrg_amt']*partB_df['fraudulent']
partB_df['fraud_average_medicare_payment_amt'] = partB_df['average_medicare_payment_amt']*partB_df['fraudulent']

In [ ]:
# Aggregate by state
state_df = partB_df.groupby('nppes_provider_state').agg({
    'line_srvc_cnt':[('total_services_count','sum')],
    'fraud_line_srvc_cnt':[('total_fraud_services_count','sum')]
}).reset_index()

# Drop multi-index
state_df.columns = ['_'.join(col) for col in state_df.columns]
state_df.columns = ['provider_state', 'total_services_count', 'fraud_services_count']

In [ ]:
# Get % fraud
state_df['fraud_services_pct'] = state_df['fraud_services_count']/state_df['total_services_count']
state_df.head()

<a id='usmap'></a>
<a href='#menu'>[Menu]</a>

In [ ]:
np.seterr(divide = 'ignore') 

fig = go.Figure(data=go.Choropleth(
    locations=state_df['provider_state'],
    z = np.log(state_df['fraud_services_pct'].astype(float))+0.000001, #log-scale
    locationmode = 'USA-states',
    colorscale = 'Reds',
    colorbar_title = "Logged %",
    marker_line_color='white'
))

fig.update_layout(
    title_text = 'Logged Percentage of Medicare Fraudulent Service by US State (2012-2016)',
    geo_scope='usa',
)

fig.show()

Heat map shows a high concentration of fraudulent Medicare claims around the Northeast and South of the US, with Texas leading all states by a wide margin.

In [ ]:
# Get dummy variables for gender
partB_df2 = pd.concat([partB_df, pd.get_dummies(partB_df['nppes_provider_gender'])], axis=1)

In [ ]:
# Aggregate by provider type
type_df = partB_df2.groupby('provider_type').agg({
    'line_srvc_cnt':['sum'],
    'fraud_line_srvc_cnt':['sum'],
    'M':['sum'],
    'F':['sum'],
    'average_submitted_chrg_amt':['median'], #since distribution skewed right
    'average_medicare_payment_amt':['median'],
    'fraud_average_submitted_chrg_amt':['max'],
    'fraud_average_medicare_payment_amt':['max']
}).reset_index()

# Drop multi-index
type_df.columns = ['_'.join(col) for col in type_df.columns]
type_df.columns = ['provider_type', 'total_services_count', 'fraud_services_count','male_count','female_count',
                   'avg_submitted_chrg_amt','avg_medicare_payment_amt', 'fraud_avg_submitted_chrg_amt','fraud_avg_medicare_payment_amt']

In [ ]:
# Sorting
type_df = type_df.sort_values('fraud_services_count',ascending=False)[:15] #get top 15 fraudulent types
type_df = type_df.sort_values('total_services_count',ascending=True).reset_index(drop=True) #re-sort by total services

# Add some fields
type_df['non_fraud_services_count'] = type_df['total_services_count'] - type_df['fraud_services_count']
type_df['fraud_services_pct'] = (type_df['fraud_services_count']/type_df['total_services_count'])*100
type_df.head()

<a id='histogram1'></a>
<a href='#menu'>[Menu]</a>

In [ ]:
# Get 2015 data only for speed
partB_2015_df = partB_df2[partB_df2['year'] == datetime(year=2015,month=1,day=1)]
top_7_types  = type_df['provider_type'][:7].tolist()

for p_type in top_7_types:
    
    #Eliminate 0 payments then log
    x = np.log(partB_2015_df[(partB_2015_df.provider_type == p_type) & (partB_2015_df.average_submitted_chrg_amt!=0)]['average_submitted_chrg_amt'])
    fraud_x = np.log(partB_2015_df[(partB_2015_df.provider_type == p_type) & (partB_2015_df.fraud_average_submitted_chrg_amt!=0)]['fraud_average_submitted_chrg_amt'])
    
    fig,ax = plt.subplots(figsize=(12,5))
    sns.distplot(x,label='Non-fraudulent', hist=False, rug=False)
    sns.distplot(fraud_x,label='Fraudulent', hist=False, rug=False)

    ax.set(
        title  ='Distribution of Submitted Charge Amount - Fraud vs. Non-fraud - '+ p_type,
        xlabel = 'Log of USD Payment Amount',
        ylabel = 'Count'
          )
    
plt.show()

No obvious evidence to say that the average fraudulent submitted charge is more expensive than non-fraudulent charge across the Top 15 Fraudulent categories, as shown in histogram comparisons.

In [ ]:
for p_type in top_7_types:
    #Eliminate 0 payments
    x = partB_2015_df[(partB_2015_df.provider_type == p_type) & (partB_2015_df.average_medicare_payment_amt!=0)]['average_medicare_payment_amt']
    fraud_x = partB_2015_df[(partB_2015_df.provider_type == p_type) & (partB_2015_df.fraud_average_medicare_payment_amt!=0)]['fraud_average_medicare_payment_amt']
    
    fig,ax = plt.subplots(figsize=(12,5))
    sns.distplot(x,label='Non-fraudulent', hist=False, rug=False)
    sns.distplot(fraud_x,label='Fraudulent', hist=False, rug=False)
    ax.set(
        title  ='Distribution of Medicare Payment Amount - Fraud vs. Non-fraud - '+ p_type,
        xlabel = 'Log of USD Payment Amount',
        ylabel = 'Count'
          )
    plt.show()

Same conclusion can be made here about Medicare Payment amount.

<a id='correlation1'></a>
<a href='#menu'>[Menu]</a>

In [ ]:
partB_2015_df = partB_df[partB_df['year'] == datetime(year=2015,month=1,day=1)]

fig, ax = plt.subplots(figsize=(15,7))

sns.heatmap(partB_2015_df.corr(method='pearson'), annot=True, fmt='.4f', 
            cmap=plt.get_cmap('coolwarm'), cbar=False, robust=True)
ax.set_yticklabels(ax.get_yticklabels(), rotation="horizontal")
ax.set(
    title  ='Correlation Heatmap in 2015 (only Continuous Variables)',

      )
plt.show()

No insights are particularly interesting here, outside of the correlations we would normally expect.

In [ ]:
# Get categorical variables (except location)
for col in ['nppes_provider_gender','provider_type']:
    partB_2015_df = pd.concat([partB_2015_df, pd.get_dummies(partB_2015_df[col], drop_first= True)], axis=1)
    partB_2015_df = partB_2015_df.drop(col, 1)

In [ ]:
def get_redundant_pairs(df):
    '''Get duplicate pairs to drop in correlation matrix after unstacking'''
    pairs_to_drop = set()
    cols = df.columns
    for i in range(0, df.shape[1]):
        for j in range(0, i+1):
            pairs_to_drop.add((cols[i], cols[j]))
    return pairs_to_drop

def get_correlations(df):
    '''Get biggest correlations'''
    au_corr = df.corr().unstack() #unstack
    labels_to_drop = get_redundant_pairs(df)
    au_corr = au_corr.drop(labels=labels_to_drop)
    return au_corr

#get relevant columns
cols = partB_2015_df.columns[20:].tolist() + ['line_srvc_cnt','bene_unique_cnt','average_submitted_chrg_amt','fraudulent']
corr = get_correlations(partB_2015_df[cols])

<a id='correlation2'></a>
<a href='#menu'>[Menu]</a>

In [ ]:
corr = corr.to_frame().reset_index() #to frame
corr.columns = ['Variable A', 'Variable B', 'Correlation']

def color_df(value):
    '''Color red if positive, green if negative'''
    if value < 0:
        color = 'red'
    elif value > 0:
        color = 'green'
    else:
        color = 'black'
    return 'color: %s' % color

corr = corr.reindex(corr['Correlation'].abs().sort_values(ascending=False).index).reset_index(drop=True)
corr[:15].style.applymap(color_df, subset=['Correlation'])

<a id='barchart1'></a>
<a href='#menu'>[Menu]</a>

In [ ]:
fig = go.Figure()

col_layout_dict = {'non_fraud_services_count': ['Non-fraud Service','rgba(50, 171, 96, 0.6)'],
                 'fraud_services_count': ['Fraud Service','rgb(255, 0, 0)']} #dict for layout

for col in ['non_fraud_services_count','fraud_services_count']:
    fig.add_trace(go.Bar(
        y=type_df['provider_type'],
        x=type_df[col],
        name=col_layout_dict[col][0],
        marker=dict(
            color=col_layout_dict[col][1],
        ),
        orientation='h',
    ))

fig.update_layout(
    barmode = 'stack',
    title = 'Top 15 Fraudulent Medicare Category by Service Count',
    paper_bgcolor='white',
    plot_bgcolor='white',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.95],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.95],
    ),
    xaxis_title_text='Count',
)

annotations = [] #annotate with %

x   = type_df['fraud_services_count']+type_df['non_fraud_services_count']+100000000
y_p = np.round(type_df['fraud_services_pct'].tolist(), decimals=2)

for y_p, x, y in zip(y_p,x,type_df['provider_type']):
    annotations.append(dict(xref='x1', yref='y1',
                            y=y, x=x,
                            text=str(y_p) + '%',
                            font=dict(family='Arial', size=12,
                                      color='rgb(255, 0, 0)'),
                            showarrow=False))

annotations.append(dict(xref='paper', yref='paper',
                        x=-0.2, y=-0.209,
                        text='Combined CMS and LEIE data' +
                             'to label the leading Fraudulent physician categories (15 Feb 2020)',
                        font=dict(family='Arial', size=10, color='rgb(150,150,150)'),
                        showarrow=False))

fig.update_layout(annotations=annotations)

fig.show()

<a id='barchart2'></a>
<a href='#menu'>[Menu]</a>

In [ ]:
fig = go.Figure()

#dict for layout
col_layout_dict = {'female_count': ['Female Physicians ','#ffcdd2'],
                 'male_count': ['Male Physicians','#A2D5F2']}

for col in ['female_count','male_count']:
    fig.add_trace(go.Bar(
        y=type_df['provider_type'],
        x=type_df[col],
        name=col_layout_dict[col][0],
        marker=dict(
            color=col_layout_dict[col][1],
        ),
        orientation='h',
    ))

fig.update_layout(
    barmode = 'stack',
    title = 'Top 15 Fraudulent Medicare Category by Service Count and Gender',
    paper_bgcolor='white',
    plot_bgcolor='white',
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=True,
        domain=[0, 0.90],
    ),
    xaxis=dict(
        zeroline=False,
        showline=False,
        showticklabels=True,
        showgrid=True,
        domain=[0, 0.90],
    ),
    xaxis_title_text='Count',
)

fig.show()

Clinical Laboratory and Ambulance Service Provider has a lot of line services but require fewer doctors?

<a href='#menu'>[Menu]</a>
<a id='train'></a>

### 5. Prep Data for Training

- Aggregate data following paper's method
- Normalize predictors
- One hot encoding
- Write data for training

In [ ]:
partB_df.columns

In [ ]:
#drop variables that we are not using
partB_train_df = partB_df.drop(columns=[
    'fraud_average_medicare_payment_amt', #for visual only
    'fraud_average_submitted_chrg_amt', #for visual only
    'fraud_line_srvc_cnt', #for visual only
    'year',
    'excl_year',
    'hcpcs_drug_indicator',
    'place_of_service',
    'nppes_provider_city',
    'nppes_provider_country',
    'hcpcs_description',
    'nppes_provider_zip'
])

In [ ]:
display(partB_train_df.head())
print("We have 9 features, including 4 categorical variables in \n {0}".format(partB_train_df.columns[1:5].tolist()))

In [ ]:
a = partB_train_df.fraudulent.value_counts()
display(a)
print('Fraudulent physicians are: {0}% of all data'.format(str(np.round((a[1]/a[0])*100,decimals = 6)))) 

In [ ]:
# One-hot encoding
for col in ['provider_type', 'nppes_provider_state', 'hcpcs_code', 'nppes_provider_gender']:
    partB_train_df = pd.concat([partB_train_df, pd.get_dummies(partB_train_df[col], drop_first= True)], axis=1)
    partB_train_df = partB_train_df.drop(col, 1) #drop column that's been encoded

In [ ]:
display(agg_partB_df.shape)
print('There are {0} predictors.'.format(agg_partB_df.shape[1]-1)) 

<a href='#menu'>[Menu]</a>
<a id='norm'></a>

In [197]:
#NOT USING
# Normalize predictors to [0,1] min-max scale
from sklearn import preprocessing

def scale_predictors(df):
    '''Takes in df and returns normalized data [0,1] '''
    x  = df.values #numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    return pd.DataFrame(x_scaled, columns=df.columns, index=df.index)

# scl_predict_df = scale_predictors(partB_train_df.iloc[:,7:(partB_train_df.shape[1]-1)])

# # merge back in with npi and label
# partB_train_df['npi']  = partB_train_df['npi_'].astype(str)
# partB_train_df['year'] = partB_train_df['year_'].astype(str).str[:4]

# partB_train_df = pd.concat([partB_train_df.iloc[:,[0,1,2,6,37]], scl_predict_df], axis=1)

<a href='#menu'>[Menu]</a>
<a id='ros'></a>

In [ ]:
# Sample data 
num_rows = 3200  # --> 20000 rows ~ 100 MB (limit on AutoAI)
percent_fraud = 0.2  # TODO: Change this value to increase the percentage of AutoAI sample that is fraud 
n_fraud = min(int(num_rows * percent_fraud), agg_partB_df_fraud_filtered['fraudulent_mean'].count())
random_state = np.random.RandomState(seed=0)
auto_ai_df = pd.concat([agg_partB_df_fraud_filtered.sample(n=n_fraud, random_state=random_state),
                       agg_partB_df_not_fraud_filtered.sample(n=(num_rows-n_fraud), random_state=random_state)])
print(f'Sample breakdown: {n_fraud} ({100*(n_fraud/num_rows):.2f}%) Fraud & {num_rows-n_fraud} ({100*((num_rows-n_fraud)/num_rows):.2f}%) Not Fraud')
auto_ai_df.head()